In [1]:
import os
import sys
os.environ["THEANO_FLAGS"] = "device=gpu"
sys.path.append(os.getcwd()+"/..")
from tasks import check

Using gpu device 0: Tesla K20Xm (CNMeM is disabled, CuDNN 3007)
/home/mcherti/work/code/scikit-learn/sklearn/cross_validation.py:42: DeprecationWarning: This module has been deprecated in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.19.
  "This module will be removed in 0.19.", DeprecationWarning)


Couldn't import dot_parser, loading of dot files will not be possible.


In [ ]:
from model import model50, model51, model51, model52

In [ ]:
all_layers = model52(nb_filters=128, w=32, h=32, c=3)

In [ ]:
import theano.tensor as T
import theano
from lasagne import layers

X_scale1 = T.tensor4()
X_scale2 = T.tensor4()
X_scale3 = T.tensor4()
X_scale4 = T.tensor4()

X_scale1_out = T.tensor4()
X_scale2_out = T.tensor4()
X_scale3_out = T.tensor4()
X_scale4_out = T.tensor4()


input_tensors = [X_scale1, X_scale2, X_scale3, X_scale4]
output_tensors = [X_scale1_out, X_scale2_out, X_scale3_out, X_scale4_out]

input_layers = ["scale1_input", "scale2_input", "scale3_input", "scale4_input"]
output_layers = ["scale1_output", "scale2_output", "scale3_output", "scale4_output"]

inputs = {all_layers[l]: t for t, l in zip(input_tensors, input_layers)}
outputs = map(lambda name:all_layers[name], output_layers)

output_pred_tensors = layers.get_output(outputs, inputs)
lambdas = [1, 1, 1, 1]
loss = 0
for outp, outp_pred, lambda_ in zip(output_tensors, output_pred_tensors, lambdas):
    loss += ((outp_pred - outp) ** 2).mean() * lambda_

In [ ]:
from lasagne import updates
import numpy as np
params = layers.get_all_params(all_layers.values(), trainable=True)
print("Number of params : {}".format(layers.count_params(all_layers.values(), trainable=True))) 
lr = theano.shared(np.array(0.1, dtype=theano.config.floatX))
params_updates = updates.adam(loss, params, learning_rate=lr)

In [ ]:
from lasagnekit.misc.draw_net import draw_to_file
from IPython.display import SVG
draw_to_file(layers.get_all_layers(all_layers['scale1_output']), 'out.svg')
SVG('out.svg')

In [ ]:
train_fn = theano.function(input_tensors + output_tensors, loss, updates=params_updates, on_unused_input='warn')

In [ ]:
predict_fn = theano.function(input_tensors, output_pred_tensors, on_unused_input='warn')

In [ ]:
from data import load_data
from lasagnekit.datasets.rescaled import Rescaled
from lasagnekit.easy import iterate_minibatches
from lasagnekit.datasets.imagecollection import ImageCollection
from lasagnekit.datasets.transformed import Transformed
w, h = 32, 32
scale = {1: 32, 2: 16, 3: 8, 4: 4}
batch_size = 128

c = 3
folder = "{}/icons".format(os.getenv("DATA_PATH"))
data_scale1 = ImageCollection(size=(w, h), nb=1535, mode='all', folder=folder)
data_scale2 = Rescaled(data_scale1, (16, 16))
data_scale3 = Rescaled(data_scale2, (8, 8))
data_scale4 = Rescaled(data_scale3, (4, 4))

data_scale2_up = Rescaled(data_scale2, (32, 32))
data_scale3_up = Rescaled(data_scale3, (16, 16))
data_scale4_up = Rescaled(data_scale4, (8, 8))

def preprocess(x, s=32):
    x = x.reshape((x.shape[0], s, s, c))
    x = x.transpose((0, 3, 1, 2))
    return x
data_scale4.load()

data_scale2_up.load()
data_scale3_up.load()
data_scale4_up.load()

In [ ]:
lr.set_value(0.01)
loss_list = []
for i in range(10000):
    
    
    X_scale1_out = preprocess(data_scale1.X, s=scale[1])
    X_scale2_out = preprocess(data_scale2.X, s=scale[2])
    X_scale3_out = preprocess(data_scale3.X, s=scale[3])
    X_scale4_out = preprocess(data_scale4.X, s=scale[4])   
    
    X_scale1 = preprocess(data_scale2_up.X, s=scale[1]) 
    X_scale2 = preprocess(data_scale3_up.X, s=scale[2]) 
    X_scale3 = preprocess(data_scale4_up.X, s=scale[3]) 
    X_scale4 = X_scale4_out
    
    total_loss = 0.
    for mini_batch in iterate_minibatches(len(data_scale1.X), batch_size):
        loss_val = train_fn(X_scale1[mini_batch], 
                            X_scale2[mini_batch], 
                            X_scale3[mini_batch], 
                            X_scale4[mini_batch],
                            X_scale1_out[mini_batch], 
                            X_scale2_out[mini_batch], 
                            X_scale3_out[mini_batch], 
                            X_scale4_out[mini_batch])
        total_loss += loss_val
    print(total_loss)
    loss_list.append(total_loss)
plt.plot(loss_list)

In [ ]:
out1, out2, out3, out4 = predict_fn(X_scale1, X_scale2, X_scale3, X_scale4)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from lasagnekit.misc.plot_weights import dispims_color
fig = plt.figure(figsize=(10, 10))
img = dispims_color(out1[0:100].transpose((0, 2, 3, 1)),border=1)
plt.imshow(img, interpolation='none')

In [ ]:
ss1 = np.random.uniform(size=(100, c, 32, 32))
ss2 = np.random.uniform(size=(100, c, 16, 16))
ss3 = np.random.uniform(size=(100, c, 8, 8))
ss4 = np.random.uniform(size=(100, c, 4, 4))

ss1 = ss1.astype(np.float32)
ss2 = ss2.astype(np.float32)
ss3 = ss3.astype(np.float32)
ss4 = ss4.astype(np.float32)


for i in range(100):
    ss1, ss2, ss3, ss4 = predict_fn(ss1, ss2, ss3, ss4)
    
%matplotlib inline
import matplotlib.pyplot as plt
from lasagnekit.misc.plot_weights import dispims_color
fig = plt.figure(figsize=(10, 10))
img = dispims_color(ss1[0:100].transpose((0, 2, 3, 1)),border=1)
plt.axis('off')
plt.imshow(img, interpolation='none')